# SQL VIEWS

# 1. Intro

a. Upload csv files to database folder. \
b. Run load_data_sqlite.py\
c. explore, clean and prepare data in table_exploration_cleaning folder.
d. clean and prepare all the individual datasets, aggregated event tables to member-level tables, created new features, removed duplicates and uploaded ready tables to clean_db.sqlite.

Now we will proceed with table joins!

# 2. Load Data

In [1]:
# Run pip install ipython-sql
%load_ext sql

%sql sqlite:///database/clean_db.sqlite

# Run the next line if error
%config SqlMagic.style = '_DEPRECATED_DEFAULT' 

%sql SELECT name from sqlite_master where type='table';

 * sqlite:///database/clean_db.sqlite
Done.


name
member_account_info
member_agreements
member_checkins
member_class_features
member_finances
member_PT
member_pos


In [2]:
%sql select * from member_account_info limit 1;

 * sqlite:///database/clean_db.sqlite
Done.


member_id,gender,birthday,age
ABC1000,male,2004-02-10 00:00:00,21.0


In [3]:
%sql select * from member_agreements limit 1;

 * sqlite:///database/clean_db.sqlite
Done.


member_id,agreement_type,start_date,cancel_date,cancel_reason,status,tenure_days,eligible_105,churn_by_105
ABC1000,Standard Annual,2024-08-21 00:00:00,None,None,Active,378,1,0


In [4]:
%sql select * from member_checkins limit 1;

 * sqlite:///database/clean_db.sqlite
Done.


member_id,total_checkins,active_months,avg_checkins_per_month,most_frequent_day,most_common_hour,last_checkin_date,days_since_last_checkin,at_risk
ABC1609,111,10,11.1,Saturday,17,2025-06-13 00:00:00,82,1


In [5]:
%sql select * from member_class_features limit 1;

 * sqlite:///database/clean_db.sqlite
Done.


member_id,total_classes,favorite_class,favorite_day,favorite_hour
ABC1239,3,spin class,Monday,15


In [6]:
%sql select * from member_finances limit 1;

 * sqlite:///database/clean_db.sqlite
Done.


member_id,monthly_fee,payment_status_tendency,total_payments,lifetime_outstanding_balance
ABC1204,45.0,on-time,1,0.0


In [7]:
%sql select * from member_PT limit 1;

 * sqlite:///database/clean_db.sqlite
Done.


member_id,total_pt_sessions,favorite_pt_session,favorite_day,favorite_hour,latest_session
ABC1859,2,hiit,Saturday,6,2024-06-15 00:00:00


In [8]:
%sql select * from member_pos limit 1;

 * sqlite:///database/clean_db.sqlite
Done.


member_id,total_items_bought,total_purchase_value,last_purchase_date
ABC1311,1,5.0,2024-06-25 00:00:00


In [ ]:
# Run this query in all tables to show basic summary
%sql SELECT \
    (SELECT COUNT(*) FROM member_pos) AS row_count, \
    (SELECT COUNT(DISTINCT member_id) FROM member_pos) AS unique_member_count, \
    (SELECT COUNT(*) FROM pragma_table_info('member_pos')) AS column_count, \
    (SELECT COUNT(*) - COUNT(DISTINCT member_id) FROM member_pos) AS duplicate_rows

 * sqlite:///database/clean_db.sqlite
Done.


row_count,unique_member_count,column_count,duplicate_rows
900,900,4,0


# Table Joins 
### Table Info

| Table Name            | Row Count | Unique MemberID Count | Column Count | duplicated_rows |
|-----------------------|-----------|-----------------------|--------------|-----------------|
| member_account_info   | 1000      | 1000                  | 4            | 0               |
| member_agreements     | 1000      | 1000                  | 7            | 0               |
| member_checkins       | 1000      | 1000                  | 10           | 0               |
| member_class_features | 810       | 810                   | 5            | 0               |
| member_finances       | 1000      | 1000                  | 5            | 0               |
| member_PT             | 139       | 139                   | 6            | 0               |
| member_pos            | 900       | 900                   | 4            | 0               |

# VIEW 
### create parent_member_view from member_account_info & member_agreements

In [10]:
%%sql 
CREATE VIEW parent_member_view as 
SELECT 
    a.member_id, 
    a.gender, a.age, 
    b.agreement_type, 
    b.start_date, 
    b.cancel_date, 
    b.cancel_reason, 
    b.status, 
    b.tenure_days,
    b.eligible_105,
    b.churn_by_105
FROM member_account_info a
LEFT JOIN member_agreements b 
    ON a.member_id = b.member_id

 * sqlite:///database/clean_db.sqlite
Done.


[]

# VIEW
### create parent_checkins from parent_member_view & member_check_ins

In [11]:
%%sql 
CREATE VIEW parent_checkins as 
SELECT 
    a.member_id, 
    a.gender, a.age, 
    a.agreement_type, 
    a.start_date, 
    a.cancel_date, 
    a.cancel_reason, 
    a.status, 
    a.tenure_days,
    a.eligible_105,
    a.churn_by_105,

    b.total_checkins, 
    b.active_months, 
    b.avg_checkins_per_month, 
    b.most_frequent_day, 
    b.most_common_hour, 
    b.last_checkin_date, 
    b.days_since_last_checkin, 
    b.at_risk

FROM parent_member_view a
LEFT JOIN member_checkins b 
    ON a.member_id = b.member_id

 * sqlite:///database/clean_db.sqlite
Done.


[]

# VIEW
### create parent_classes from parent_member_view & member_class_features

In [12]:
%%sql 
CREATE VIEW parent_classes as 
SELECT 
    a.member_id, 
    a.gender, a.age, 
    a.agreement_type, 
    a.start_date, 
    a.cancel_date, 
    a.cancel_reason, 
    a.status, 
    a.tenure_days,
    a.eligible_105,
    a.churn_by_105,

    b.total_classes,
    b.favorite_class,
    b.favorite_day,
    b.favorite_hour

FROM parent_member_view a
LEFT JOIN member_class_features b
    ON a.member_id = b.member_id

 * sqlite:///database/clean_db.sqlite
Done.


[]

# VIEW
### create parent_finances from parent_member_view & member_finances

In [13]:
%%sql 
CREATE VIEW parent_finances as 
SELECT 
    a.member_id, 
    a.gender, a.age, 
    a.agreement_type, 
    a.start_date, 
    a.cancel_date, 
    a.cancel_reason, 
    a.status, 
    a.tenure_days,
    a.eligible_105,
    a.churn_by_105,

    b.monthly_fee,
    b.payment_status_tendency,
    b.total_payments,
    b.lifetime_outstanding_balance

FROM parent_member_view a
LEFT JOIN member_finances b
    ON a.member_id = b.member_id

 * sqlite:///database/clean_db.sqlite
Done.


[]

# VIEW
### create parent_pt from parent_member_view & member_PT

In [14]:
%%sql 
CREATE VIEW parent_pt as 
SELECT 
    a.member_id, 
    a.gender, a.age, 
    a.agreement_type, 
    a.start_date, 
    a.cancel_date, 
    a.cancel_reason, 
    a.status, 
    a.tenure_days,
    a.eligible_105,
    a.churn_by_105,

    b.member_id,
    b.total_pt_sessions,
    b.favorite_pt_session,
    b.favorite_day,
    b.favorite_hour,
    b.latest_session

FROM parent_member_view a
LEFT JOIN member_PT b
    ON a.member_id = b.member_id

 * sqlite:///database/clean_db.sqlite
Done.


[]

# VIEW
### create parent_pos from parent_member_view & member_pos

In [15]:
%%sql 
CREATE VIEW parent_pos as 
SELECT 
    a.member_id, 
    a.gender, a.age, 
    a.agreement_type, 
    a.start_date, 
    a.cancel_date, 
    a.cancel_reason, 
    a.status, 
    a.tenure_days,
    a.eligible_105,
    a.churn_by_105,

    b.member_id,
    b.total_items_bought,
    b.total_purchase_value,
    b.last_purchase_date

FROM parent_member_view a
LEFT JOIN member_pos b
    ON a.member_id = b.member_id

 * sqlite:///database/clean_db.sqlite
Done.


[]